### **Computer Vision Multi-Face Blink & Yawn Counter using Color-Coded MediaPipe Mesh.**

In [6]:
import cv2
import mediapipe as mp
import numpy as np
from scipy.spatial import distance as dist
import time

# CONFIGURATION
EAR_THRESHOLD = 0.21     # Eye closed threshold
MAR_THRESHOLD = 0.09     # Mouth open threshold

Right_Eye = [33, 160, 158, 133, 153, 144]
Left_Eye  = [362, 385, 387, 263, 373, 380]

Lip_Points = [13, 14]    # Upper + lower lip points

# Text properties (GLOBAL → avoids UnboundLocalError)
FONT = cv2.FONT_HERSHEY_SIMPLEX
FONT_SCALE = 0.6
THICKNESS = 2

# HELPER FUNCTIONS
def ear(pts):
    A = dist.euclidean(pts[1], pts[5])
    B = dist.euclidean(pts[2], pts[4])
    C = dist.euclidean(pts[0], pts[3])
    return (A + B) / (2.0 * C) if C != 0 else 0.0


def normalized_lip_distance(u, l):
    u = np.array([u.x, u.y])
    l = np.array([l.x, l.y])
    return np.linalg.norm(u - l)


def pixel_landmarks(face, w, h):
    return [(int(lm.x * w), int(lm.y * h)) for lm in face.landmark]


def face_box(face, w, h):
    xs = [lm.x for lm in face.landmark]
    ys = [lm.y for lm in face.landmark]
    return int(min(xs)*w), int(min(ys)*h), int(max(xs)*w), int(max(ys)*h)

# MAIN FACE DETECTION FUNCTION
def start_detection():

    mp_face_mesh = mp.solutions.face_mesh
    mp_draw = mp.solutions.drawing_utils
    draw_spec = mp_draw.DrawingSpec(color=(0, 255, 255), thickness=1, circle_radius=1)

    counters = {}     # Per face blink/mouth count
    p_time = 0        # For FPS

    cv2.namedWindow("Multi-Face Mesh EAR + MAR", cv2.WINDOW_NORMAL)
    cap = cv2.VideoCapture(0)

    with mp_face_mesh.FaceMesh(
        max_num_faces=10,
        refine_landmarks=True,
        min_detection_confidence=0.7,
        min_tracking_confidence=0.7
    ) as mesh:

        while True:
            ok, frame = cap.read()
            if not ok:
                continue

            frame = cv2.flip(frame, 1)
            h, w, _ = frame.shape

            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = mesh.process(rgb)

            # FACE PROCESSING
            if results.multi_face_landmarks:
                for face_id, face in enumerate(results.multi_face_landmarks):

                    # Draw full mesh
                    mp_draw.draw_landmarks(
                        frame,
                        face,
                        mp_face_mesh.FACEMESH_TESSELATION,
                        draw_spec,
                        draw_spec
                    )

                    # Create counter bucket for NEW face_id
                    if face_id not in counters:
                        counters[face_id] = {
                            "L": 0, "R": 0, "M": 0,
                            "prev_L": 1, "prev_R": 1, "prev_M": 0
                        }

                    pts = pixel_landmarks(face, w, h)
                    x1, y1, x2, y2 = face_box(face, w, h)

                    # -------- EYE EAR --------
                    RE = [pts[i] for i in Right_Eye]
                    LE = [pts[i] for i in Left_Eye]

                    rEAR = ear(RE)
                    lEAR = ear(LE)

                    # Left blink
                    if lEAR < EAR_THRESHOLD and counters[face_id]["prev_L"] == 1:
                        counters[face_id]["L"] += 1

                    # Right blink
                    if rEAR < EAR_THRESHOLD and counters[face_id]["prev_R"] == 1:
                        counters[face_id]["R"] += 1

                    counters[face_id]["prev_L"] = 1 if lEAR > EAR_THRESHOLD else 0
                    counters[face_id]["prev_R"] = 1 if rEAR > EAR_THRESHOLD else 0

                    # -------- MOUTH MAR --------
                    upper = face.landmark[Lip_Points[0]]
                    lower = face.landmark[Lip_Points[1]]
                    mar = normalized_lip_distance(upper, lower)

                    if mar > MAR_THRESHOLD and counters[face_id]["prev_M"] == 0:
                        counters[face_id]["M"] += 1
                        counters[face_id]["prev_M"] = 1

                    if mar <= MAR_THRESHOLD:
                        counters[face_id]["prev_M"] = 0

                    # -------- DRAW POINTS --------
                    for p in RE + LE:
                        cv2.circle(frame, p, 2, (0, 0, 255), -1)

                    cv2.circle(frame, pts[Lip_Points[0]], 3, (255, 100, 0), -1)
                    cv2.circle(frame, pts[Lip_Points[1]], 3, (255, 100, 0), -1)

                    # -------- TEXT OUTPUT --------
                    cv2.putText(frame,
                                f"FACE {face_id} - Avg EAR: {((rEAR + lEAR) / 2):.2f}",
                                (x1, y1 - 10),
                                FONT, 0.6, (0, 255, 0), 2)

                    cv2.putText(frame,
                                f"L-Blinks: {counters[face_id]['L']} / R-Blinks: {counters[face_id]['R']}",
                                (x1, y2 + 20),
                                FONT, FONT_SCALE, (0, 0, 255), THICKNESS)

                    cv2.putText(frame,
                                f"Mouth Opens: {counters[face_id]['M']} (MAR: {mar:.2f})",
                                (x1, y2 + 45),
                                FONT, FONT_SCALE, (0, 255, 255), THICKNESS)
            # FPS DISPLAY
            c_time = time.time()
            fps = 1 / (c_time - p_time) if c_time != p_time else 0
            p_time = c_time

            cv2.putText(frame, f"FPS: {int(fps)}",
                        (10, 30), FONT, 0.7, (255, 0, 0), 2)

            cv2.imshow("Multi-Face Mesh EAR + MAR", frame)

            if cv2.waitKey(5) & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()

# RUN
if __name__ == "__main__":
    start_detection()


## 🔬 Features Included in the Code


### 1. **Multi-Face Detection and Tracking** 

* **Description:** The system uses `mp.solutions.face_mesh` with `max_num_faces=10` to simultaneously detect and process up to ten faces in the video feed. It uses a dictionary (`counters`) indexed by `face_id` to **independently track events** (blinks, mouth opens) for each person, maintaining persistence across frames.

### 2. **Real-Time Blink Detection (Eye Aspect Ratio - EAR)**

* **Description:** It calculates the **Eye Aspect Ratio (EAR)** using six specific landmarks for both the right and left eyes. The EAR, calculated by the `ear(pts)` helper function, provides a numerical measure of eye openness.
* **Counting Accuracy:** The core counting logic uses a **falling edge detection** (`lEAR < EAR_THRESHOLD and counters[face_id]["prev_L"] == 1`) to accurately count a blink event only when the eye transitions from an **open state** to a **closed state**, ensuring a single count per full blink.

### 3. **Real-Time Mouth Open/Yawn Detection (Normalized Lip Distance - MAR)** 

* **Description:** It calculates the vertical distance between two key lip landmarks (13 and 14) in **normalized coordinates** (renamed to **MAR** in the code, standing for *Mouth Aspect Ratio* or distance).
* **Counting Accuracy:** The counting logic uses a **rising edge detection** (`mar > MAR_THRESHOLD and counters[face_id]["prev_M"] == 0`) to accurately count an opening event (yawn, speaking, etc.) only when the mouth transitions from a **closed state** to an **open state**, ensuring a single count per event.

### 4. **Multi-Color Face Mesh Visualization** 

* **Description:** This feature replaces a simple bounding box with the detailed **MediaPipe Face Mesh** using `mp_drawing.draw_landmarks`. Crucially, it assigns a **unique color** from the `FACE_COLORS` list to the mesh lines and points of **each detected face** (`current_face_color = FACE_COLORS[face_id % len(FACE_COLORS)]`). This visually separates and tracks individuals in the frame.

### 5. **Enhanced Text Display and Information Overlay** 

* **Description:** It uses `cv2.putText` to overlay event counts and metrics on the video feed. The display is designed for clarity, grouping related statistics and placing them near the respective face's bounding box.
    * **Face-Specific Metrics:** Displays the `FACE ID` and average **EAR value** (e.g., `Avg EAR: 0.28`).
    * **Grouped Counts:** Consolidates left/right blink counts and mouth open counts into separate, clear lines, and uses the face's **unique color** for the text associated with that face.

### 6. **Real-Time Frame Rate (FPS) Monitor** 

* **Description:** A simple performance monitoring feature that calculates and displays the **Frames Per Second (FPS)** in the top-left corner. It measures the time difference between consecutive frames using the `time` library, providing immediate feedback on the application's processing speed.

### 7. **Video Handling and Preprocessing** 

* **Description:** Includes standard video capture and manipulation:
    * **Camera Initialization:** Uses `cv2.VideoCapture(0)` to access the primary webcam.
    * **Mirroring:** Uses `cv2.flip(frame, 1)` to flip the image horizontally, providing a mirrored view that is more intuitive for the user.
    * **Color Conversion:** Converts the frame from BGR (OpenCV default) to RGB for processing by the MediaPipe model (`cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)`).